In [ ]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import random
from scipy.special import softmax

In [ ]:
df = pd.read_csv('mental_health.csv', encoding='latin1')

In [ ]:
df.head()

,text,label
0,dear american teens question dutch person hear...,0
1,nothing look forward lifei dont many reasons k...,1
2,music recommendations im looking expand playli...,0
3,im done trying feel betterthe reason im still ...,1
4,worried year old girl subject domestic physic...,1


In [ ]:
df.shape

(27977, 2)

In [ ]:
def get_length(text):
    return len(text.split(' '))

In [ ]:
df['length']=df['text'].apply(get_length)

In [ ]:
df=df[df['length']>300]

In [ ]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess(text):
    text = text.translate(str.maketrans("", "", string.punctuation)).lower()
    words = [word for word in text.split(' ') if word not in stopwords.words('english')]
    return ' '.join(words)

In [ ]:
df['text']=df['text'].apply(preprocess)

In [ ]:
model = transformers.pipeline(
    'sentiment-analysis',
    model="yikuan8/Clinical-BigBird",
    tokenizer="yikuan8/Clinical-BigBird",
    return_all_scores=True
)

Some weights of the model checkpoint at yikuan8/Clinical-BigBird were not used when initializing BigBirdForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at yikuan8/

In [ ]:
ground_truth=[]
for i in range(100):
    ground_truth.append(df['label'].iloc[i])

In [ ]:
pred=[]
for i in range(100):
    print('Completed '+str(i)+' of '+str(931))
    label0=model.predict(df['text'].iloc[i])[0][0]['score']
    label1=model.predict(df['text'].iloc[i])[0][1]['score'] 
    if label1>label0:
        pred.append(1)
    else:
        pred.append(0)

Attention type 'block_sparse' is not possible if sequence_length: 325 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Completed 0 of 931
Completed 1 of 931
Completed 2 of 931
Completed 3 of 931
Completed 4 of 931
Completed 5 of 931
Completed 6 of 931
Completed 7 of 931
Completed 8 of 931
Completed 9 of 931
Completed 10 of 931
Completed 11 of 931
Completed 12 of 931
Completed 13 of 931
Completed 14 of 931
Completed 15 of 931
Completed 16 of 931
Completed 17 of 931
Completed 18 of 931
Completed 19 of 931
Completed 20 of 931
Completed 21 of 931
Completed 22 of 931
Completed 23 of 931
Completed 24 of 931
Completed 25 of 931
Completed 26 of 931
Completed 27 of 931
Completed 28 of 931
Completed 29 of 931
Completed 30 of 931
Completed 31 of 931
Completed 32 of 931
Completed 33 of 931
Completed 34 of 931
Completed 35 of 931
Completed 36 of 931
Completed 37 of 931
Completed 38 of 931
Completed 39 of 931
Completed 40 of 931
Completed 41 of 931
Completed 42 of 931
Completed 43 of 931
Completed 44 of 931
Completed 45 of 931
Completed 46 of 931
Completed 47 of 931
Completed 48 of 931
Completed 49 of 931
Completed 

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_true=ground_truth,y_pred=pred)

0.79

In [ ]:
pd.DataFrame(classification_report(y_true=ground_truth,y_pred=pred,output_dic=True))

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'              precision    recall  f1-score   support\n\n           0       0.26      1.00      0.42       244\n           1       0.00      0.00      0.00       687\n\n    accuracy                           0.26       931\n   macro avg       0.13      0.50      0.21       931\nweighted avg       0.07      0.26      0.11       931\n'